# Linear elasticity in a multipatch domain

In [24]:
%pylab inline
import scipy
import itertools

from pyiga import bspline, assemble, vform, geometry, vis, solvers

import numpy as np
import matplotlib.pyplot as plt

# assemble matrix
from scipy.sparse import coo_matrix, block_diag, bmat

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


C:\Users\MHeig\anaconda3\lib\site-packages\IPython\core\magics\pylab.py:162: UserWarning: pylab import has clobbered these variables: ['f', 'bmat', 'rc']
`%matplotlib` prevents importing * from pylab and numpy
  warn("pylab import has clobbered these variables: %s"  % clobbered +


In [25]:
p = 2  # spline degree 
n_el = (50, 10)   # number of elements in y/x direction, isotropic material
#n_el = (50, 20)   # number of elements in y/x direction
# displacement space: degree p,   continuity p-1
kvs = tuple(bspline.make_knots(p, 0.0, 1.0, n, mult=2) for n in n_el) # or : mult=2

# Taylor-Hood discretization space
kvs_u = tuple(bspline.make_knots(p,   0.0, 1.0, n, mult=2) for n in n_el)
kvs_p = tuple(bspline.make_knots(p-1, 0.0, 1.0, n, mult=1) for n in n_el)

m_u = tuple(kv.numdofs for kv in kvs)


In [26]:
# define geometry map

geos = [
    geometry.quarter_annulus(),
    geometry.quarter_annulus().rotate_2d(3*pi/2),
    geometry.quarter_annulus().rotate_2d(pi),
    geometry.quarter_annulus().rotate_2d(pi/2)
]

patches = [(kvs, g) for g in geos]
patches_u = [(kvs_u, g) for g in geos]
patches_p = [(kvs_p, g) for g in geos]

patches= patches_u

#print(kvs)

In [27]:
for g in geos:
    vis.plot_geo(g)
axis('scaled');

In [28]:
# rubber, Lamé-coeff.
Mu = 0.003 # [GPa]
Lam = 0.00028416 #  [GPa] (under ass. ny=0.48, almost incompressible)

In [29]:
def A11_vf(dim):
    from pyiga.vform import VForm, grad, inner, div, dx, Dx
    vf = VForm(dim)
    u, v = vf.basisfuns()
    
    grad_uv= inner(grad(u), grad(v))
    div_11 = Dx(u,0) * Dx(v,0)
    
    vf.add(Mu * ( grad_uv +  div_11) * dx )
    return vf

def A12_vf(dim):
    from pyiga.vform import VForm, grad, inner, div, dx, Dx
    vf = VForm(dim)
    u, v = vf.basisfuns()
    
    div_21 = Dx(u,0) * Dx(v,1)
    
    vf.add(Mu * div_21 * dx )
    return vf

           
def A21_vf(dim):
    from pyiga.vform import VForm, grad, inner, div, dx, Dx
    vf = VForm(dim)
    u, v = vf.basisfuns()

    grad_uv= inner(grad(u), grad(v))
    div_12 = Dx(u,1) * Dx(v,0)
    
    vf.add( Mu * div_12 * dx)
    return vf
           
def A22_vf(dim):
    from pyiga.vform import VForm, grad, inner, div, dx, Dx
    vf = VForm(dim)
    u, v = vf.basisfuns()

    grad_uv= inner(grad(u), grad(v))
    div_22 = Dx(u,1) * Dx(v,1)
    
    vf.add(Mu * ( grad_uv +  div_22) * dx )
    return vf

def B1_vf(dim):
    from pyiga.vform import VForm, dx, Dx
    vf = VForm(dim)
    u, p = vf.basisfuns()
    vf.add( - Dx(u,0) * p * dx)
    return vf

def B2_vf(dim):
    from pyiga.vform import VForm, dx, Dx
    vf = VForm(dim)
    u, p = vf.basisfuns()
    vf.add( - Dx(u,1) * p * dx)
    return vf

def C_vf(dim):
    from pyiga.vform import VForm, dx
    vf = VForm(dim)
    p,q= vf.basisfuns()
    vf.add( -(1/Lam) * p * q * dx)
    return vf


In [30]:
# source term f 
f = lambda x,y: (0.0) # source term

In [31]:
#Neumann BC
def g_Nx(x, y): return (1.5e-4) # x dir in coordinate system
def g_Ny(x, y): return (-1.5e-4) # y dir

# set inhom. Neumann cond. on distal end, hom. NBC elsewhere
neu_data_x = [
    #(0, 'top', g),
    #(0,'right', g_N),
    #(1,'right', g_N),
    #(2,'right', g_N) 
    (3,'right', g_Nx) 
]
neu_data_y = [
    #(0, 'top', g),
    #(0,'right', g_N),
    #(1,'right', g_N),
    #(2,'right', g_N) 
    (3,'right', g_Ny) 
]

MP_x = assemble.Multipatch(patches_u, {'N':neu_data_x}, automatch=True) # X
MP_y = assemble.Multipatch(patches_u, {'N':neu_data_y}, automatch=True) # Y
MP_p = assemble.Multipatch(patches_p, {}, automatch=True) # p...pressure

#MP = assemble.Multipatch(patches, automatch=True) # without Neumann! 

In [32]:
#A, rhs, N = MP.assemble_system(linelast_vf(2), vform.L2functional_vf(2, physical=True), f=f)

A11, rhs1, nbc11 = MP_x.assemble_system(A11_vf(2), vform.L2functional_vf(2, physical=True), f=f)
A12, rhs12, nbc12 = MP_x.assemble_system(A12_vf(2), vform.L2functional_vf(2, physical=True), f=f) 
A21, rhs21, nbc21 = MP_y.assemble_system(A21_vf(2), vform.L2functional_vf(2, physical=True), f=f) 
A22, rhs2, nbc22 = MP_y.assemble_system(A22_vf(2), vform.L2functional_vf(2, physical=True), f=f)

C, rc, nc = MP_p.assemble_system(C_vf(2), vform.L2functional_vf(2, physical=True), f=f)

print(shape(A11))
print(shape(C))

(8400, 8400)
(2200, 2200)


In [33]:
#B1, rb1, nb1 = MP_x.assemble_system(B1_vf(2), vform.L2functional_vf(2, physical=True), f=f)
#B2, rb2, nb2 = MP_y.assemble_system(B2_vf(2), vform.L2functional_vf(2, physical=True), f=f)

In [34]:
# assemble single patch
n = MP_x.numdofs
m = MP_p.numdofs
B = scipy.sparse.csr_matrix((m, n))
print(shape(B))
B1 = scipy.sparse.csr_matrix((m, n))
B2 = scipy.sparse.csr_matrix((m, n))

(2200, 8400)


In [35]:
for p in range(MP_x.numpatches):
    kvs_x, geo_x = MP_x.patches[p]
    kvs_y, geo_y = MP_y.patches[p]
            
    Xx= MP_x.patch_to_global(p)
    Xy= MP_y.patch_to_global(p)
    X_xy= bmat([[Xx, Xy]])
    X_xx= bmat([[Xx, Xx]])
    X_yy= bmat([[Xy, Xy]])
    X_p= MP_p.patch_to_global(p)

    
    #B1_p = assemble.assemble(' - Dx(u,0) * p * dx ', (kvs_x, kvs_p), args=None, bfuns=[('u',1,0), ('p',1,1)],geo=geo_x)
    #B2_p = assemble.assemble(' - Dx(u,1) * p * dx ', (kvs_y, kvs_p), args=None, bfuns=[('u',1,0), ('p',1,1)],geo=geo_y)


    #B1 += X_p @ B1_p @ Xx.T
    #B2 += X_p @ B2_p @ Xy.T
  

    B_p = assemble.assemble(' div(u) * p * dx', (kvs_u, kvs_p), args=None, bfuns=[('u',2,0), ('p',1,1)],geo=geo_x)
    #
    B += X_p @ B_p @ X_xy.T

    

print(shape(Xx))
print(shape(X_p))
print(shape(B_p))
print(shape(B))
#print(shape(B1_p))
#print(shape(B1))

(8400, 2121)
(2200, 561)
(561, 4242)
(2200, 8400)


In [39]:
#A = bmat([[A11, A12, B1.T], [A12.T, A22, B2.T], [B1, B2, C]])
A = bmat([[A11, A12, B.T], [A12.T, A22, B.T], [B, B, C]])
print(shape(A))

(19000, 19000)


In [40]:
#print(shape(A11)[0])

In [41]:
rhs= np.hstack((rhs1, rhs2)) # F= (fx,fy) 
rhs = np.pad(rhs, (0,shape(C)[0])) # (F,G)
print(shape(rhs))

#combine Neumann-BC to appropriate shape
nbc= np.hstack((nbc11 , nbc22)) # x- and y- dir
#nbc= np.hstack((np.zeros(shape(nbc11)), nbc22,)) ## in y-dir only
nbc = np.pad(nbc, (0,shape(C)[0]))
print(shape(nbc))

(19000,)
(19000,)


In [42]:
# Dirichlet BC
# Dirichlet boundary function g
def g_zero(x, y): return (0.0) 

# set up Dirichlet boundary conditions
# vectorize bcs
bcs = MP_x.compute_dirichlet_bcs_vec([
    (1, 'right', g_zero)
])

print(shape(bcs))
#print(bc)

(2, 202)


In [43]:
# set up and solve linear system
LS = assemble.RestrictedLinearSystem(A, rhs + nbc, bcs) #
u_sol = solvers.make_solver(LS.A, spd=False).dot(LS.b)
u = LS.complete(u_sol)

#print(shape(u))
print(u)


[ 4.64491796e-02  4.65238157e-02  4.64436239e-02 ...  5.76490404e-07
 -3.25848569e-06 -1.52200872e-05]


In [44]:
np.linalg.norm(LS.A*u_sol-LS.b) # residual

1.0482834977183162e-15

In [45]:
#print(MP.numdofs) # global dofs (=all dofs interface dofs,i.e. 2x16-4 = 28)

u1 = u[:MP_x.numdofs]
u2 = u[MP_x.numdofs:2*MP_x.numdofs]

#print('u1= ',u1)
#print(shape(u1))
#print('u2= ', u2)
#print(shape(u2))


In [46]:
len(u)

19000

In [47]:
MP_x.numdofs*2

16800

In [48]:
# restrict solution to each individual patch
u1_funcs = [geometry.BSplineFunc(kvs, MP_x.global_to_patch(p) @ u1)
           for p in range(len(patches))]
u2_funcs = [geometry.BSplineFunc(kvs, MP_y.global_to_patch(p) @ u2)
           for p in range(len(patches))]


In [50]:
# set refinement steps for grid (visualization)
ref= 10

# evaluate displacement over a grid in the parameter domain
xgrid = linspace(0, 1, ref)
xygrid = (xgrid, xgrid)

figsize(10,10)
fig, ax = plt.subplots()
i=0
             
for (u1_func, u2_func, (kvs, geo)) in zip(u1_funcs, u2_funcs, patches): #u_funcs 
    #print(i)
    dis1= u1_func.grid_eval(xygrid)
    dis2 = u2_func.grid_eval(xygrid) 
    dis = np.stack((dis1,dis2), axis=-1)
    #print('dis=', dis)
    #print('dis1= ', dis1)
    #print(shape(dis1))
    #print('dis2= ', dis2)
    i+=1
    
    vis.plot_deformation(dis, ref, geo, ax, vmin=0.0, vmax=1.25e-4)
    
    
colorbar();
axis('equal')


(-2.202348628929946, 2.04697257859892, -2.046972578599023, 2.202348628929951)